In [1]:
!pip install -q roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.3 MB/s eta 0:00:00


In [2]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from roboflow import Roboflow
from sklearn.metrics import average_precision_score

# Parameters
API_KEY = ""#<--------------------------------- Replace this
img_size = (224, 224)
batch_size = 32
merged_dir = "/content/merged_dataset"
AUTOTUNE = tf.data.AUTOTUNE


In [3]:
datasets_info = [
    ("zaghamshamsi", "soil-classification", 1),
    ("ai-usztq", "toprak-verim-analizi", 1),
    ("demian-a5xbv", "soil-analysis-55sc2", 1)
]

os.makedirs(merged_dir, exist_ok=True)
rf = Roboflow(api_key=API_KEY)

for workspace, project_name, version_number in datasets_info:
    project = rf.workspace(workspace).project(project_name)
    version = project.version(version_number)
    dataset = version.download("folder")

    for split in ["train", "valid", "test"]:
        split_path = os.path.join(dataset.location, split)
        if not os.path.exists(split_path):
            continue

        for class_folder in os.listdir(split_path):
            class_path = os.path.join(split_path, class_folder)
            if os.path.isdir(class_path):
                dest_class_path = os.path.join(merged_dir, class_folder)
                os.makedirs(dest_class_path, exist_ok=True)

                for file in os.listdir(class_path):
                    src_file = os.path.join(class_path, file)
                    dst_file = os.path.join(dest_class_path, f"{workspace}_{project_name}_{file}")
                    shutil.copy(src_file, dst_file)


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Soil-Classification-1 in folder:: 100%|██████████| 4306/4306 [00:04<00:00, 949.74it/s] 


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Toprak-verim-Analizi-1 in folder:: 100%|██████████| 1163/1163 [00:00<00:00, 1973.62it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Soil-Analysis-1 in folder:: 100%|██████████| 162/162 [00:00<00:00, 1823.40it/s]


In [4]:
classes_to_keep = [
    "Clay soil", "Yellow Soil", "Black Soil", "Alluvial soil",
    "Red soil", "medium", "loam", "sandy", "heavy"
]

for class_name in os.listdir(merged_dir):
    if class_name not in classes_to_keep:
        shutil.rmtree(os.path.join(merged_dir, class_name))
        print(f"❌ Deleted: {class_name}")


❌ Deleted: Laterite soil
❌ Deleted: Clayey soils
❌ Deleted: Sandy loam
❌ Deleted: Sandy soil
❌ Deleted: Loamy soil


In [14]:
dataset_path = "/content/merged_dataset"
img_size = (224, 224)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE


In [10]:
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    seed=123,
    validation_split=0.2,
    subset="training",
    image_size=img_size,
    batch_size=batch_size
)

raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    seed=123,
    validation_split=0.2,
    subset="validation",
    image_size=img_size,
    batch_size=batch_size
)

class_names = raw_train_ds.class_names
num_classes = len(class_names)

train_ds = raw_train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.prefetch(buffer_size=AUTOTUNE)


Found 5440 files belonging to 9 classes.
Using 4352 files for training.
Found 5440 files belonging to 9 classes.
Using 1088 files for validation.


In [11]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2)
])


In [12]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model for feature extraction

model = keras.Sequential([
    keras.Input(shape=img_size + (3,)),
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,700 (16.08 MB)

 Trainable params: 165,129 (645.04 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [13]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10


KeyboardInterrupt: 

In [ ]:
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

aps = []
for class_id in range(num_classes):
    y_true_binary = (y_true == class_id).astype(int)
    y_pred_binary = (y_pred == class_id).astype(int)
    ap = average_precision_score(y_true_binary, y_pred_binary)
    aps.append(ap)

mean_ap = np.mean(aps)
print(f"mAP: {mean_ap:.4f}")


In [ ]:
base_model.trainable = True  # Unfreeze EfficientNet

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


In [ ]:
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

aps = []
for class_id in range(num_classes):
    y_true_binary = (y_true == class_id).astype(int)
    y_pred_binary = (y_pred == class_id).astype(int)
    ap = average_precision_score(y_true_binary, y_pred_binary)
    aps.append(ap)

mean_ap = np.mean(aps)
print(f"mAP after Fine-Tuning: {mean_ap:.4f}")
